## Введение
Линейная регрессия - один из наиболее хорошо изученных методов машинного обучения, позволяющий прогнозировать значения количественного признака в виде линейной комбинации прочих признаков с параметрами - весами модели. Оптимальные (в смысле минимальности некоторого функционала ошибки) параметры линейной регрессии можно найти аналитически с помощью нормального уравнения или численно с помощью методов оптимизации.  

Линейная регрессия использует простой функционал качества - среднеквадратичную ошибку. Мы будем работать с выборкой, содержащей 3 признака. Для настройки параметров (весов) модели решается следующая задача:
$$\Large \frac{1}{\ell}\sum_{i=1}^\ell{{((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}^2} \rightarrow \min_{w_0, w_1, w_2, w_3},$$
где $x_{i1}, x_{i2}, x_{i3}$ - значения признаков $i$-го объекта, $y_i$ - значение целевого признака $i$-го объекта, $\ell$ - число объектов в обучающей выборке.

## Градиентный спуск
Параметры $w_0, w_1, w_2, w_3$, по которым минимизируется среднеквадратичная ошибка, можно находить численно с помощью градиентного спуска.
Градиентный шаг для весов будет выглядеть следующим образом:
$$\Large w_0 \leftarrow w_0 - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$
$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}},\ j \in \{1,2,3\}$$
Здесь $\eta$ - параметр, шаг градиентного спуска.

## Стохастический градиентный спуск
Проблема градиентного спуска, описанного выше, в том, что на больших выборках считать на каждом шаге градиент по всем имеющимся данным может быть очень вычислительно сложно. 
В стохастическом варианте градиентного спуска поправки для весов вычисляются только с учетом одного случайно взятого объекта обучающей выборки:
$$\Large w_0 \leftarrow w_0 - \frac{2\eta}{\ell} {((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)}$$
$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} {x_{kj}((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)},\ j \in \{1,2,3\},$$
где $k$ - случайный индекс, $k \in \{1, \ldots, \ell\}$.

## Нормальное уравнение 
Нахождение вектора оптимальных весов $w$ может быть сделано и аналитически.
Мы хотим найти такой вектор весов $w$, чтобы вектор $y$, приближающий целевой признак, получался умножением матрицы $X$ (состоящей из всех признаков объектов обучающей выборки, кроме целевого) на вектор весов $w$. То есть, чтобы выполнялось матричное уравнение:
$$\Large y = Xw$$
Домножением слева на $X^T$ получаем:
$$\Large X^Ty = X^TXw$$
Это хорошо, поскольку теперь матрица $X^TX$ - квадратная, и можно найти решение (вектор $w$) в виде:
$$\Large w = {(X^TX)}^{-1}X^Ty$$
Матрица ${(X^TX)}^{-1}X^T$ - [*псевдообратная*](https://ru.wikipedia.org/wiki/Псевдообратная_матрица) для матрицы $X$. В NumPy такую матрицу можно вычислить с помощью функции [numpy.linalg.pinv](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linalg.pinv.html).

Однако, нахождение псевдообратной матрицы - операция вычислительно сложная и нестабильная в случае малого определителя матрицы $X$ (проблема мультиколлинеарности). 
На практике лучше находить вектор весов $w$ решением матричного уравнения 
$$\Large X^TXw = X^Ty$$Это может быть сделано с помощью функции [numpy.linalg.solve](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.solve.html).

Но все же на практике для больших матриц $X$ быстрее работает градиентный спуск, особенно его стохастическая версия.

In [502]:
#stochastic gradient descent use

import pandas as pd
import numpy as np

In [507]:
def compute_error(y, w):
    error = 0
    N = len(X)
    error = (np.dot(X,w)-y.T)**2
    return np.sum(error) / N 

In [524]:

def stochastic_gradient_step(X, y, w, train_ind, eta=0.1):
    grad = (2/len(X)) * X[train_ind,:] * (np.dot(X[train_ind,:],w) - y[train_ind,:] )
    return  w - eta * grad

In [535]:
def stochastic_gradient_descent(X,y,w_init,max_iter=10000):
      
    w = w_init
    errors = []
    np.random.seed(42)
        
    for i in range(max_iter):
        random_ind = np.random.randint(X.shape[0])
        w = stochastic_gradient_step(X,y,w,random_ind)
        errors.append(compute_error(y,w))
                
    return w, errors

def linear_regression():
    #init dataset
    adver_data = pd.read_csv('advertising.csv')
    X = adver_data[['TV', 'radio', 'newspaper']].values
    y = adver_data[['sales']].values
    
    #matrix scaling
    means = np.mean(X, axis=0)
    stds = np.std(X, axis=0)
    X = (X - means) / stds
    
    X = np.hstack((np.ones((X.shape[0], 1)),X))
    w_init = np.zeros(X.shape[1])
    
    
    print(
        'Start learning at w = {0}, error = {1}'.format(
            w_init,
            compute_error(y,w_init)
        )
    )
    
    w, errors = stochastic_gradient_descent(X,y,w_init)
    print(
        'End learning at w = {0}, error = {1}'.format(
            w,
            compute_error(y,w)
        )
    )

In [536]:
%%time
linear_regression()

Start learning at w = [ 0.  0.  0.  0.], error = 223.71625
End learning at w = [ 13.95189116   3.90570603   2.82822419  -0.08254811], error = 2.7927058255015904
CPU times: user 1.33 s, sys: 8 ms, total: 1.34 s
Wall time: 1.32 s
